In [1]:
from dotenv import load_dotenv
import os
import requests
import datetime

load_dotenv()

True

## Time Information

In [2]:

API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

def get_time_info(city_name):
    """Fetch latitude, longitude, and time zone information from a city name."""
    
    # Step 1: Get Latitude & Longitude
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city_name}&key={API_KEY}"
    geocode_response = requests.get(geocode_url).json()

    if geocode_response["status"] != "OK":
        print(f"Error in Geocoding: {geocode_response['status']}")
        return None

    location = geocode_response["results"][0]["geometry"]["location"]
    lat, lon = location["lat"], location["lng"]

    # Step 2: Get Time Zone Information
    timestamp = int(datetime.datetime.utcnow().timestamp())
    timezone_url = f"https://maps.googleapis.com/maps/api/timezone/json?location={lat},{lon}&timestamp={timestamp}&key={API_KEY}"
    timezone_response = requests.get(timezone_url).json()

    if timezone_response["status"] != "OK":
        print(f"Error in Time Zone API: {timezone_response['status']}")
        return None

    time_zone_id = timezone_response["timeZoneId"]
    time_zone_name = timezone_response["timeZoneName"]
    raw_offset = timezone_response["rawOffset"] 
    dst_offset = timezone_response["dstOffset"]
    local_offset = raw_offset + dst_offset
    local_time = datetime.datetime.utcnow() + datetime.timedelta(seconds=local_offset)

    # print(f"📍 City: {city_name}")
    # print(f"🌍 Coordinates: ({lat}, {lon})")
    # print(f"🕒 Time Zone ID: {time_zone_id}")
    # print(f"⏳ Time Zone Name: {time_zone_name}")
    # print(f"⏰ Local Time: {local_time.strftime('%Y-%m-%d %H:%M:%S')}")

    return {
        "city": city_name,
        "latitude": lat,
        "longitude": lon,
        "time_zone_id": time_zone_id,
        "time_zone_name": time_zone_name,
        "local_time": local_time.strftime('%Y-%m-%d %H:%M:%S')
    }

get_time_info("Tokyo")
get_time_info("New York")
get_time_info("Athens")


{'city': 'Athens',
 'latitude': 37.9838096,
 'longitude': 23.7275388,
 'time_zone_id': 'Europe/Athens',
 'time_zone_name': 'Eastern European Standard Time',
 'local_time': '2025-03-21 13:36:14'}

## Country Info

In [3]:
import requests

def get_country_info(country_name):
    """
    Fetches country details from the RestCountries API.
    - country_name: Name of the country (e.g., "Japan", "France")
    """

    url = f"https://restcountries.com/v3.1/name/{country_name}"
    response = requests.get(url)

    if response.status_code == 200:
        country_data = response.json()[0]  # Get first match

        # Extract relevant data
        name = country_data.get("name", {}).get("common", "Unknown")
        capital = country_data.get("capital", ["Unknown"])[0]
        population = country_data.get("population", "Unknown")
        region = country_data.get("region", "Unknown")
        subregion = country_data.get("subregion", "Unknown")
        currency = list(country_data.get("currencies", {}).keys())[0] if "currencies" in country_data else "Unknown"
        languages = ", ".join(country_data.get("languages", {}).values()) if "languages" in country_data else "Unknown"
        flag = country_data.get("flags", {}).get("png", "")

        print(f"\n🌍 Country: {name}")
        print(f"🏛️ Capital: {capital}")
        print(f"👥 Population: {population}")
        print(f"🌎 Region: {region}, {subregion}")
        print(f"💰 Currency: {currency}")
        print(f"🗣️ Languages: {languages}")
        print(f"🏴 Flag: {flag}")

        return {
            "country": name,
            "capital": capital,
            "population": population,
            "region": region,
            "subregion": subregion,
            "currency": currency,
            "languages": languages,
            "flag": flag,
        }

    else:
        print(f"Error: Could not fetch data for '{country_name}'")
        return None

# Example Usage
get_country_info("Netherlands")
get_country_info("France")
get_country_info("UAE")



🌍 Country: Netherlands
🏛️ Capital: Amsterdam
👥 Population: 16655799
🌎 Region: Europe, Western Europe
💰 Currency: EUR
🗣️ Languages: Dutch
🏴 Flag: https://flagcdn.com/w320/nl.png

🌍 Country: France
🏛️ Capital: Paris
👥 Population: 67391582
🌎 Region: Europe, Western Europe
💰 Currency: EUR
🗣️ Languages: French
🏴 Flag: https://flagcdn.com/w320/fr.png

🌍 Country: United Arab Emirates
🏛️ Capital: Abu Dhabi
👥 Population: 9890400
🌎 Region: Asia, Western Asia
💰 Currency: AED
🗣️ Languages: Arabic
🏴 Flag: https://flagcdn.com/w320/ae.png


{'country': 'United Arab Emirates',
 'capital': 'Abu Dhabi',
 'population': 9890400,
 'region': 'Asia',
 'subregion': 'Western Asia',
 'currency': 'AED',
 'languages': 'Arabic',
 'flag': 'https://flagcdn.com/w320/ae.png'}

## Weather Data

In [4]:
import requests

OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')

def get_weather(city_name):
    """
    Fetches real-time weather data using OpenWeather API (Free Plan).
    """

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units=metric"

    response = requests.get(url)

    if response.status_code == 200:
        weather_data = response.json()

        city = weather_data["name"]
        country = weather_data["sys"]["country"]
        temperature = weather_data["main"]["temp"]
        humidity = weather_data["main"]["humidity"]
        condition = weather_data["weather"][0]["description"].capitalize()
        wind_speed = weather_data["wind"]["speed"]

        print(f"\n🌍 City: {city}, {country}")
        print(f"🌡️ Temperature: {temperature}°C")
        print(f"💧 Humidity: {humidity}%")
        print(f"🌤️ Condition: {condition}")
        print(f"💨 Wind Speed: {wind_speed} m/s")

        return {
            "city": city,
            "country": country,
            "temperature": temperature,
            "humidity": humidity,
            "condition": condition,
            "wind_speed": wind_speed,
        }

    else:
        print(f"Error: Could not fetch weather data for '{city_name}'")
        return None

# Example Usage
get_weather("Abu Dhabi")
get_weather("Cambridge")
get_weather("Patras")



🌍 City: Abu Dhabi, AE
🌡️ Temperature: 38.01°C
💧 Humidity: 11%
🌤️ Condition: Clear sky
💨 Wind Speed: 2.57 m/s

🌍 City: Cambridge, CA
🌡️ Temperature: -3.79°C
💧 Humidity: 80%
🌤️ Condition: Scattered clouds
💨 Wind Speed: 3.09 m/s

🌍 City: Pátrai, GR
🌡️ Temperature: 15.4°C
💧 Humidity: 55%
🌤️ Condition: Clear sky
💨 Wind Speed: 2.4 m/s


{'city': 'Pátrai',
 'country': 'GR',
 'temperature': 15.4,
 'humidity': 55,
 'condition': 'Clear sky',
 'wind_speed': 2.4}